#### -Importing necessary libraries

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime


In [28]:
# Load the dataset and preprocess
df = pd.read_csv("preprocessed_dataset.csv")
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

In [ ]:
df.head()

In [29]:
from sklearn.preprocessing import StandardScaler

# Example division of data into training, validation and test sets for the normal functioning times:
# data: data from each minute
# about 70% for training, 15% for validation, 15% for testing

# Reduce the amount of observations (with averaging)
df = df.resample("T").mean()

included_rows1 = [df.index.get_loc(datetime.datetime(2017,4,23,19,0,0)),
                    df.index.get_loc(datetime.datetime(2017,5,13,0,59,00))]
included_rows2 = [df.index.get_loc(datetime.datetime(2017,6,15,1,0,0)),
                    df.index.get_loc(datetime.datetime(2017,7,24,0,59,00))]
included_rows3 = [df.index.get_loc(datetime.datetime(2017,8,15,1,0,0)),
                    df.index.get_loc(datetime.datetime(2017,9,10,0,0,0))]

test_mins = 5
training_n = 5 # training mins = training_n*test_mins
test_start = '2017-04-23 00:00:00' # example start of test partition
test_start = datetime.datetime.strptime(test_start, '%Y-%m-%d %H:%M:%S')
flag = 0

df_array = df.to_numpy()
row_number = df.index.get_loc(test_start)

# Iterate through the data
while flag == 0:
    test_y_end = row_number+test_mins+1
    training_X_start = row_number-(training_n+1)*test_mins
    # Stay within included rows
    if ((test_y_end <= included_rows1[1] and training_X_start >= included_rows1[0])
    or (test_y_end <= included_rows2[1] and training_X_start >= included_rows2[0])
    or (test_y_end <= included_rows3[1] and training_X_start >= included_rows3[0])):

        scaler = StandardScaler()

        # Testing set
        test_X = df_array[row_number:test_y_end-1,:-1]
        test_y = df_array[row_number+1:test_y_end,-1]

        # Validation set
        validation_X = df_array[row_number-test_mins:row_number,:-1]
        validation_y = df_array[row_number-test_mins+1:row_number+1,-1]

        # Training set
        training_X = df_array[training_X_start:row_number-test_mins,:-1]
        training_y = df_array[training_X_start+1:row_number-test_mins+1,-1]

        # Lagged predictor variables and the response variable in one matrix
        test_data = np.column_stack([test_X, test_y])
        validation_data = np.column_stack([validation_X, validation_y])
        training_data = np.column_stack([training_X, training_y])

        # Compute the mean and std with training data
        scaler.fit(training_data)
        # Perform standardization
        training_data = scaler.transform(training_data)
        validation_data = scaler.transform(validation_data)
        test_data = scaler.transform(test_data)

        if test_y_end == included_rows3[1]:
            flag = 1

        row_number = row_number + 1
    else:
        if test_y_end > included_rows3[1]:
            flag = 1

        row_number = row_number + 1